# Big Data project A.Y. 2024-2025

## Members

- Giovanni Antonioni
- Luca Rubboli - 0001083742

## Second job

In [1]:
import org.apache.spark.sql.SparkSession

Intitializing Scala interpreter ...

Spark Web UI available at http://10.201.108.0:4040
SparkContext available as 'sc' (version = 3.5.1, master = local[*], app id = local-1744708804156)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession
